# Sentinel-2 DataCube supporting functions and execution examples

The primary focus of our work to date has been to demonstrate the capability to configure and analyse data other than Landsat (out primary dataset for code testing and development). The examples presented below are demonstrations only and some additional refinement of the content presented within them and the configuration they reference is anticipated.

- step 1: prepare your datasets for ingest
- step 2: prepare system for ingest 
- step 3: index prepared datasets (datasets are now accessible to DataCube)
    - L1C local file system OK - example included
    - L1C AWS               ALPHA
    - L2A local file system BETA
- step 4: ingest indexed datasets (datasets are now DataCube datasets)
    - L1C local file system OK
    - L1C AWS               #TODO 
    - L2A local file system #TODO
- step 5: load dataset spatiotemporal subset to analysis array
    - L1C local and AWS
    - L2A local
- step 6: plotting examples

## STEP 1: Prepare S2 L1C data for indexing and ingesting (writes the yaml ingest configuration)

### S2A L1C local filesystem SAFE

In [2]:
s2granule_prepare='/home/simonaoliver/code/agdc-v2/utils/s2prepare.py'
s2_l1c_safe='/home/simonaoliver/data/S2A_OPER_PRD_MSIL1C_PDMC_20151205T075851_R073_V20151205T001643_20151205T001643.SAFE'
# creation of the valid data bounds geometry is what consumes much of the execution time - can be commented out in preparation code
!python $s2granule_prepare  $s2_l1c_safe
s2_l1c_safeyaml_path='/home/simonaoliver/data/S2A_OPER_PRD_MSIL1C_PDMC_20151205T075851_R073_V20151205T001643_20151205T001643.SAFE/agdc-metadata.yaml'

2016-11-28 16:15:14,761 INFO Processing /home/simonaoliver/data/S2A_OPER_PRD_MSIL1C_PDMC_20151205T075851_R073_V20151205T001643_20151205T001643.SAFE/S2A_OPER_MTD_SAFL1C_PDMC_20151205T075851_R073_V20151205T001643_20151205T001643.xml
2016-11-28 16:16:55,694 INFO Writing 4 dataset(s) into /home/simonaoliver/data/S2A_OPER_PRD_MSIL1C_PDMC_20151205T075851_R073_V20151205T001643_20151205T001643.SAFE/agdc-metadata.yaml


### S2A on AWS S3

##### Use the S3 metadata retrieval code to find available data for a spatiotemporal range

http://sentinel-pds.s3-website.eu-central-1.amazonaws.com/

tiles/[UTM code]/latitude band/square/[year]/[month]/[day]/[sequence]/DATA

For example, the files for individual scene are available in the following location: http://sentinel-s2-l1c.s3-website.eu-central-1.amazonaws.com/#tiles/10/S/DG/2015/12/7/0/

Where:

    [UTM code] = e.g. 10 - grid zone designator.

    latitude band = e.g. S - latitude band are lettered C- X (omitting the letters "I" and "O").

    square = e.g DG - pair of letters designating one of the 100,000-meter side grid squares inside the grid zone.

    [year] = e.g. 2015 - is the year the data was collected.

    [month] = e.g. 12 - is the month of the year the data was collected (without leading zeros).

    [day] = e.g. 7 - is the day of the month the data was collected (without leading zeros).

    [sequence] = e.g. 0 - in most cases there will be only one image per day. In case there are more (in northern latitudes), the following images will be 1,2,…\.


### Identify the MGRS cell reference using this web map service

https://mappingsupport.com/p/coordinates-mgrs-google-maps.html

### update the zone, latitude_band and square identified above

In [5]:
import urllib3
def check_url(url):
    http = urllib3.PoolManager()
    r = http.request('GET', url)
    if r.status!=200:
        #print('url not found')
        return False
    else:
        #print(r.status)
        return True
urls = {}

# set the utm zone (assume south for this exercise ie. EPSG:327 vs EPSG:326[zone])
zone = 55
latitude_band = 'H'
square = 'FA'
# prepare configurations for all of the available data for a tile
# alter the temporal ranges below to optimise testing

import time
print('working',end="")
end_year = int(time.strftime("%Y"))
for year in range(2016,end_year+1):
        for month in range(11,12):
            for day in range(1,31):
                url = 'http://sentinel-s2-l1c.s3.amazonaws.com/tiles/'+str(zone)+'/'+latitude_band+'/'+square+'/'+str(year)+'/'+str(month)+'/'+str(day)+'/0/tileInfo.json'
                sceneid=((url.split('tiles/')[1]).replace('/','.')).replace('.tileInfo.json','')
                print('.',end="")
                if check_url(url):
                    urls[sceneid] = url             


working..............................

### print the dictionary of urls for the target MGRS tile

In [6]:
urls

{'55.H.FA.2016.11.16.0': 'http://sentinel-s2-l1c.s3.amazonaws.com/tiles/55/H/FA/2016/11/16/0/tileInfo.json',
 '55.H.FA.2016.11.19.0': 'http://sentinel-s2-l1c.s3.amazonaws.com/tiles/55/H/FA/2016/11/19/0/tileInfo.json',
 '55.H.FA.2016.11.26.0': 'http://sentinel-s2-l1c.s3.amazonaws.com/tiles/55/H/FA/2016/11/26/0/tileInfo.json',
 '55.H.FA.2016.11.6.0': 'http://sentinel-s2-l1c.s3.amazonaws.com/tiles/55/H/FA/2016/11/6/0/tileInfo.json',
 '55.H.FA.2016.11.9.0': 'http://sentinel-s2-l1c.s3.amazonaws.com/tiles/55/H/FA/2016/11/9/0/tileInfo.json'}

In [7]:
s2granule_AWS_prepare='/home/simonaoliver/code/agdc-v2/utils/s2awsprepare.py'

### take a look at the data using rasterio library - AGDCv2 uses this under the hood

In [9]:
import rasterio
rasterio_s2open_test = rasterio.open('http://sentinel-s2-l1c.s3.amazonaws.com/tiles/55/H/FA/2016/11/6/0/B01.jp2')

In [10]:
rasterio_s2open_test.bounds

BoundingBox(left=600000.0, bottom=5990200.0, right=709800.0, top=6100000.0)

### loop through the url dict
### prepare each of the target tiles for ingest
### write the yaml configuration locally with an explicit path to help DataCube decode the target file

In [11]:
for key in urls.keys():
    print(urls[key])
    output_yaml = key+'.yaml'
    tileinfo = urls[key]
    !python $s2granule_AWS_prepare $tileinfo --output $output_yaml


http://sentinel-s2-l1c.s3.amazonaws.com/tiles/55/H/FA/2016/11/9/0/tileInfo.json
http://sentinel-s2-l1c.s3.amazonaws.com/tiles/55/H/FA/2016/11/16/0/tileInfo.json
http://sentinel-s2-l1c.s3.amazonaws.com/tiles/55/H/FA/2016/11/19/0/tileInfo.json
http://sentinel-s2-l1c.s3.amazonaws.com/tiles/55/H/FA/2016/11/6/0/tileInfo.json
http://sentinel-s2-l1c.s3.amazonaws.com/tiles/55/H/FA/2016/11/26/0/tileInfo.json


## Sen2Cor output Surface Reflectance (SR) and Scene Classification (SC)

### prepare an example Sen2Cor dataset. 
### note the performance issues related to format and drivers: http://www.gdal.org/frmt_sentinel2.html

In [12]:
s2granule_sen2cor_prepare='/home/simonaoliver/code/agdc-v2/utils/sen2cor_prepare.py'
sen2cor_safe = '/media/simonaoliver/datacube/input/S2_NBAR/S2A_USER_PRD_MSIL2A_PDMC_20160314T110945_R073_V20160314T001209_20160314T001209.SAFE'
sen2cor_safe_yaml = '/media/simonaoliver/datacube/input/S2_NBAR/S2A_USER_PRD_MSIL2A_PDMC_20160314T110945_R073_V20160314T001209_20160314T001209.SAFE/agdc-metadata.yaml'

!python $s2granule_sen2cor_prepare $sen2cor_safe --output $sen2cor_safe_yaml

2016-11-28 16:31:45,327 INFO Processing /media/simonaoliver/datacube/input/S2_NBAR/S2A_USER_PRD_MSIL2A_PDMC_20160314T110945_R073_V20160314T001209_20160314T001209.SAFE/S2A_USER_MTD_SAFL2A_PDMC_20160314T110945_R073_V20160314T001209_20160314T001209.xml
2016-11-28 16:31:51,785 INFO Writing 13 dataset(s) into /media/simonaoliver/datacube/input/S2_NBAR/S2A_USER_PRD_MSIL2A_PDMC_20160314T110945_R073_V20160314T001209_20160314T001209.SAFE/agdc-metadata.yaml


### define the dataset type (product definition - includes S2 L1C and Sen2Cor index support)

In [13]:
s2granule_dataset_type='/home/simonaoliver/code/agdc-v2/docs/config_samples/dataset_types/s2_granules.yaml'
sen2corgranule_dataset_type='/home/simonaoliver/code/agdc-v2_20161031/docs/config_samples/dataset_types/s2_granules_SCL_only.yaml'

# STEP 2: Prepare System for ingest
## Add the S2 products to the AGDC database

In [14]:
# Remove old datacube db instances if they exist and create a new
!dropdb datacube
!createdb datacube
# initialise the database to prepare to product addition
!datacube system init
# add the product configurations to enable indexing of prepared data
!datacube -v product add $s2granule_dataset_type
!datacube system check

Initialising database...
Created.
Checking indexes/views.
Done.
2016-11-28 17:41:25,545 datacube INFO Running datacube command: /home/simonaoliver/venvs/py35/bin/datacube -v product add /home/simonaoliver/code/agdc-v2/docs/config_samples/dataset_types/s2_granules.yaml
2016-11-28 17:41:28,327 datacube.index.postgres._dynamic INFO Creating index: dix_s2a_level1c_granule_lat_lon_time
2016-11-28 17:41:28,400 datacube.index.postgres._dynamic INFO Creating index: dix_s2a_level1c_granule_time_lat_lon
2016-11-28 17:41:28,501 datacube.index.postgres._dynamic INFO Creating index: dix_s2a_level1c_granule_sat_path_sat_row_time
2016-11-28 17:41:28,640 datacube.index.postgres._dynamic INFO Creating index: dix_s2a_level1c_granule_orbit
Added "s2a_level1c_granule"
2016-11-28 17:41:34,242 datacube.index.postgres._dynamic INFO Creating index: dix_s2a_sen2cor_granule_lat_lon_time
2016-11-28 17:41:34,332 datacube.index.postgres._dynamic INFO Creating index: dix_s2a_sen2cor_granule_time_lat_lon
2016-11-28 

# STEP 3: Index the prepared datasets
## S2 datasets to the AGDC database - this supports on-the-fly datacube creation - datacube can decode the underlying dataset

### Index the AWS datasets

In [15]:
for key in urls.keys():
    output_yaml = key+'.yaml'
    !datacube -v dataset add $output_yaml --auto-match

2016-11-28 17:42:54,098 datacube INFO Running datacube command: /home/simonaoliver/venvs/py35/bin/datacube -v dataset add 55.H.FA.2016.11.9.0.yaml --auto-match
2016-11-28 17:42:54,709 datacube-dataset INFO Matched Dataset <id=8a5d0519-3a62-5015-b954-db32b3c59841 type=s2a_level1c_granule location=/home/simonaoliver/code/testing_notebooks/55.H.FA.2016.11.9.0.yaml>
2016-11-28 17:42:54,714 datacube.index._datasets INFO Indexing 8a5d0519-3a62-5015-b954-db32b3c59841
2016-11-28 17:43:05,849 datacube INFO Running datacube command: /home/simonaoliver/venvs/py35/bin/datacube -v dataset add 55.H.FA.2016.11.16.0.yaml --auto-match
2016-11-28 17:43:06,365 datacube-dataset INFO Matched Dataset <id=da542cb3-1a00-57b2-911d-f773f987ce8d type=s2a_level1c_granule location=/home/simonaoliver/code/testing_notebooks/55.H.FA.2016.11.16.0.yaml>
2016-11-28 17:43:06,441 datacube.index._datasets INFO Indexing da542cb3-1a00-57b2-911d-f773f987ce8d
2016-11-28 17:43:16,988 datacube INFO Running datacube command: /hom

### Index the local L1C datasets

In [16]:
!datacube -v dataset add $s2_l1c_safeyaml_path --auto-match

2016-11-28 17:43:56,597 datacube INFO Running datacube command: /home/simonaoliver/venvs/py35/bin/datacube -v dataset add /home/simonaoliver/data/S2A_OPER_PRD_MSIL1C_PDMC_20151205T075851_R073_V20151205T001643_20151205T001643.SAFE/agdc-metadata.yaml --auto-match
2016-11-28 17:43:56,909 datacube-dataset INFO Matched Dataset <id=dafe3d91-362d-470d-98f2-594356a404fa type=s2a_level1c_granule location=/home/simonaoliver/data/S2A_OPER_PRD_MSIL1C_PDMC_20151205T075851_R073_V20151205T001643_20151205T001643.SAFE/agdc-metadata.yaml>
2016-11-28 17:43:56,910 datacube.index._datasets INFO Indexing dafe3d91-362d-470d-98f2-594356a404fa
2016-11-28 17:43:57,028 datacube-dataset INFO Matched Dataset <id=55516399-df8f-4231-9b34-075ab821fc00 type=s2a_level1c_granule location=/home/simonaoliver/data/S2A_OPER_PRD_MSIL1C_PDMC_20151205T075851_R073_V20151205T001643_20151205T001643.SAFE/agdc-metadata.yaml>
2016-11-28 17:43:57,029 datacube.index._datasets INFO Indexing 55516399-df8f-4231-9b34-075ab821fc00
2016-11-

### Index sen2cor datasets

In [17]:
!datacube -v dataset add $sen2cor_safe_yaml --auto-match

2016-11-28 17:50:35,300 datacube INFO Running datacube command: /home/simonaoliver/venvs/py35/bin/datacube -v dataset add /media/simonaoliver/datacube/input/S2_NBAR/S2A_USER_PRD_MSIL2A_PDMC_20160314T110945_R073_V20160314T001209_20160314T001209.SAFE/agdc-metadata.yaml --auto-match
2016-11-28 17:50:35,935 datacube-dataset INFO Matched Dataset <id=e0411b2a-74e2-4aeb-91fb-9bfbe22ffdde type=s2a_sen2cor_granule location=/media/simonaoliver/datacube/input/S2_NBAR/S2A_USER_PRD_MSIL2A_PDMC_20160314T110945_R073_V20160314T001209_20160314T001209.SAFE/agdc-metadata.yaml>
2016-11-28 17:50:35,938 datacube.index._datasets INFO Indexing e0411b2a-74e2-4aeb-91fb-9bfbe22ffdde
2016-11-28 17:50:36,462 datacube-dataset INFO Matched Dataset <id=be8bb19a-36ec-427d-88e0-9658dcce1c06 type=s2a_sen2cor_granule location=/media/simonaoliver/datacube/input/S2_NBAR/S2A_USER_PRD_MSIL2A_PDMC_20160314T110945_R073_V20160314T001209_20160314T001209.SAFE/agdc-metadata.yaml>
2016-11-28 17:50:36,528 datacube.index._datasets IN

# STEP 4: Ingest the indexed datasets
## The above indexes the source input dataset - now we can ingest to a DataCube dataset NetCDF4.CF1.6 as described by the ingester configuration file

In [ ]:
s2aingester10 = '/home/simonaoliver/code/agdc-v2/docs/config_samples/ingester/s2amsil1c_albers_10.yaml'
!datacube -v ingest --config-file $s2aingester10

In [29]:
!datacube -v ingest --help


2016-11-28 22:08:17,419 datacube INFO Running datacube command: /home/simonaoliver/venvs/py35/bin/datacube -v ingest --help
Usage: datacube ingest [OPTIONS]

  Ingest datasets

Options:
  -c, --config-file PATH       Ingest configuration file
  --year INTEGER RANGE
  --queue-size INTEGER RANGE   Task queue size
  --save-tasks PATH            Save tasks to the specified file
  --load-tasks PATH            Load tasks from the specified file
  -d, --dry-run                Check if everything is ok
  --executor <CHOICE TEXT>...  Run parallelized, either locally or distrbuted.
                               eg:
                               --executor multiproc 4 (OR)
                               --executor
                               distributed 10.0.0.8:8888
  -h, --help                   Show this message and exit.


# STEP 5: Load a spatiotemporal subset of the ingested data

In [18]:
%matplotlib inline
from matplotlib import pyplot as plt
import datacube
from datacube.model import Range
from datetime import datetime
dc = datacube.Datacube(app='dc-example')
from datacube.storage import masking
from datacube.storage.masking import mask_valid_data as mask_invalid_data
import pandas
import xarray
import numpy

In [19]:
import folium
from IPython.display import display
import geopandas
from shapely.geometry import mapping
from shapely.geometry import MultiPolygon
import rasterio
import shapely.geometry
import shapely.ops
from functools import partial
import pyproj
from datacube.model import CRS

In [20]:
def datasets_union(dss, inputcrs):
    thing = shapely.ops.unary_union([shapely.geometry.Polygon(ds.extent.points) for ds in dss])
    return shapely.geometry.shape(rasterio.warp.transform_geom(inputcrs,'EPSG:4326',
                                        shapely.geometry.mapping(thing)))

In [21]:
import random
def plot_folium(shapes):

    mapa = folium.Map(location=[-30,150], zoom_start=4)
    colors=['#00ff00', '#ff0000', '#00ffff', '#ffffff', '#000000', '#ff00ff']
    for shape in shapes:
        style_function = lambda x: {'fillColor': '#000000' if x['type'] == 'Polygon' else '#00ff00', 
                                   'color' : random.choice(colors)}
        poly = folium.features.GeoJson(mapping(shape), style_function=style_function)
        mapa.add_children(poly)
    display(mapa)

In [22]:
def plot_rgb(image, fake_saturation):
    image = mask_invalid_data(image)
    rgb = image.to_array(dim='color')
    rgb = rgb.transpose(*(rgb.dims[1:]+rgb.dims[:1]))  # make 'color' the last dimension
    rgb = rgb.where((rgb <= fake_saturation).all(dim='color'))  # mask out pixels where any band is 'saturated'
    rgb /= fake_saturation  # scale to [0, 1] range for imshow

    rgb.plot.imshow(x=image.crs.dimensions[1], y=image.crs.dimensions[0],
                col='time', col_wrap=5, add_colorbar=False)

### List the indexable products

In [23]:
dc.list_products()

,name,description,product_type,instrument,sat_row,platform,time,sat_path,lon,format,lat,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,,,
1,s2a_level1c_granule,Sentinel-2 Level 1 - Ortho Rectified,S2MSI1C,MSI,None,SENTINEL_2A,None,None,None,JPEG2000,None,NaN,NaN,NaN,NaN
2,s2a_sen2cor_granule,Sentinel-2 Level 2 - Sen2Cor Bottom of Atmosph...,S2MSI2Ap,MSI,None,SENTINEL_2A,None,None,None,JPEG2000,None,NaN,NaN,NaN,NaN


### List the available measurements for each product

In [24]:
dc.list_measurements()

aliases   dtype  \
product             measurement                                   
s2a_level1c_granule 01            [band_01, B01, Band1]  uint16   
                    02            [band_02, B02, Band2]  uint16   
                    03            [band_03, B03, Band3]  uint16   
                    04            [band_04, B04, Band4]  uint16   
                    05            [band_05, B05, Band5]  uint16   
                    06            [band_06, B06, Band6]  uint16   
                    07            [band_07, B07, Band7]  uint16   
                    08            [band_08, B08, Band8]  uint16   
                    8A           [band_8A, B8A, Band8A]  uint16   
                    09            [band_09, B09, Band9]  uint16   
                    10           [band_10, B10, Band10]  uint16   
                    11           [band_11, B11, Band11]  uint16   
                    12           [band_12, B12, Band12]  uint16   
s2a_sen2cor_granule B01_60m                         NaN  uint16   
                    B02_10m                         NaN  uint16   
                    B02_20m                         NaN  uint16   
                    B02_60m                         NaN  uint16   
                    B03_10m       [band_03, B03, Band3]  uint16   
                    B03_20m                         NaN  uint16   
                    B03_60m                         NaN  uint16   
                    B04_10m                         NaN  uint16   
                    B04_20m                         NaN  uint16   
                    B04_60m                         NaN  uint16   
                    B05_20m                         NaN  uint16   
                    B05_60m                         NaN  uint16   
                    B06_20m                         NaN  uint16   
                    B06_60m                         NaN  uint16   
                    B07_20m                         NaN  uint16   
                    B07_60m                         NaN  uint16   
                    B08_10m                         NaN  uint16   
                    B8A_20m                         NaN  uint16   
                    B8A_60m                         NaN  uint16   
                    B09_60m                         NaN  uint16   
                    B11_20m                         NaN  uint16   
                    B11_60m                         NaN  uint16   
                    B12_20m                         NaN  uint16   
                    B12_60m                         NaN  uint16   
                    SCL_20m                         NaN  uint16   

                                                                  flags_definition  \
product             measurement                                                      
s2a_level1c_granule 01                                                         NaN   
                    02                                                         NaN   
                    03                                                         NaN   
                    04                                                         NaN   
                    05                                                         NaN   
                    06                                                         NaN   
                    07                                                         NaN   
                    08                                                         NaN   
                    8A                                                         NaN   
                    09                                                         NaN   
                    10                                                         NaN   
                    11                                                         NaN   
                    12                                                         NaN   
s2a_sen2cor_granule B01_60m                                                  

## Show the spatial extent of the ingested data

In [26]:
plot_folium([datasets_union(dc.index.datasets.search_eager(product='s2a_l1c_albers'),'EPSG:3577')])

ValueError: Input is not a valid geometry object

In [27]:
plot_folium([datasets_union(dc.index.datasets.search_eager(product='s2a_level1c_granule'),'EPSG:32755')])

### Load spatiotemporal range of available L1C data - don't care where it is sitting (on AWS S3 or locally)

In [28]:
# list the available indexed L1C datasets - noting both AWS and local file system datasets
!datacube dataset search product=s2a_level1c_granule

id,product,location
8a5d0519-3a62-5015-b954-db32b3c59841,s2a_level1c_granule,file:///home/simonaoliver/code/testing_notebooks/55.H.FA.2016.11.9.0.yaml
da542cb3-1a00-57b2-911d-f773f987ce8d,s2a_level1c_granule,file:///home/simonaoliver/code/testing_notebooks/55.H.FA.2016.11.16.0.yaml
8be4c6eb-8e28-522d-a768-d4a16cb54be0,s2a_level1c_granule,file:///home/simonaoliver/code/testing_notebooks/55.H.FA.2016.11.19.0.yaml
2b186db7-da68-5dc0-b55e-320f5e890e63,s2a_level1c_granule,file:///home/simonaoliver/code/testing_notebooks/55.H.FA.2016.11.6.0.yaml
d6b1a882-c8ab-5984-8cbc-b75323a639c5,s2a_level1c_granule,file:///home/simonaoliver/code/testing_notebooks/55.H.FA.2016.11.26.0.yaml
dafe3d91-362d-470d-98f2-594356a404fa,s2a_level1c_granule,file:///home/simonaoliver/data/S2A_OPER_PRD_MSIL1C_PDMC_20151205T075851_R073_V20151205T001643_20151205T001643.SAFE/agdc-metadata.yaml
55516399-df8f-4231-9b34-075ab821fc00,s2a_level1c_granule,file:///home/simonaoliver/data/S2A_OPER_PRD_MSIL1C_PDMC_20151205T075851_R0

In [ ]:
s2a_l1c = dc.load(product='s2a_level1c_granule',y=(6030000, 6031000), x=(662000, 663000), crs='EPSG:32755', measurements=['04','03','02'], output_crs='EPSG:4326', resolution=(-0.00025,0.00025))

In [ ]:
s2a_l1c_scl = dc.load(product='s2a_level1c_granule_scl', output_crs='EPSG:4326', resolution=(-0.00025,0.00025))

In [ ]:
s2a_l1c

In [ ]:
plot_rgb(s2a_l1c,5000)

## Write to example output NetCDF

In [ ]:
import os
from datacube.storage.storage import write_dataset_to_netcdf
netcdf_write_example = '/home/simonaoliver/example4.nc'
if os.path.isfile(netcdf_write_example):
    os.remove(netcdf_write_example)
    
write_dataset_to_netcdf(s2a_l1c, netcdf_write_example)

## Read back as xarray

In [ ]:
import xarray

In [ ]:
example = xarray.open_dataset('/home/simonaoliver/example2.nc')

In [ ]:
example['02'].plot()

In [ ]:
s2a_l1c['02'].plot()

In [ ]:
s2a_l1c_albers_10m = dc.load(product='s2a_level1c_albers_10', x=(147.36, 147.41), y=(-35.1, -35.15), measurements=['red','green','blue'])

In [ ]:
plot_rgb(s2a_l1c_albers_10m, 6000)

In [ ]:
s2a_sen2cor_60m = dc.load(product='s2a_sen2cor_granule_scl',x=(147.36, 147.41), y=(-37.1, -37.15), measurements=['SCL_20m'], output_crs='EPSG:4326', resolution=(-0.0060, 0.0060))

In [ ]:
sen2corSLC = dc.load(product='s2a_sen2cor_granule_scl', output_crs='EPSG:4326', resolution=(-0.0025, 0.0025))

In [ ]:
s2a_sen2cor_60m